# Scaling Sensitivity Analysis with Emulation

## Context
In the field of cardiovascular modeling, capturing the dynamics of blood flow and the associated pressures and volumes within the vascular system is crucial for understanding heart function and disease. Physics-based models that accurately represent these dynamics often require significant computational resources, making them challenging to apply in large-scale or real-time scenarios. Emulation techniques provide a way to achieve high-fidelity simulations of the cardiovascular system, allowing for efficient and accurate analysis of key hemodynamic parameters.

## Purpose
In this tutorial, we will demonstrate the application of `AutoEmulate` within a cardiac modeling pipeline, specifically focusing on the reconstruction of critical heart parameters using synthetic data. The goal is to demonstrate how these measures—such as arterial systemic pressure, and flow rates —can be leveraged to estimate a range of model parameters related to cardiac mechanics and cardiovascular hemodynamics. 

By the end of this tutorial, participants will have a clear understanding of how to use emulation to facilitate the real-time simulation of complex cardiac dynamics on a local CPU.

Participants will learn how to explore the relationship between underlying cardiac health parameters and non-invasive measures using synthetic data, with a focus on how these insights can improve understanding and estimation of key cardiovascular dynamics.

## Description
This tutorial includes:

1. setting up a simulation : The system simulated is a tube with an input flow rate at any given time. The tube is divided to 10 compartments which allows for the study of the pressure and flow rate at various points in the tube. 
2. Running the simulation for 60 sets of parameters sampled from the parameter space.  
3. applying Autoemulate to find the best emulator fot this simulation 
5. assessing model accuracy, performing sensitivity analysis, and comparing computational resources for different modeling approaches. 

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from tqdm import tqdm

from autoemulate.compare import AutoEmulate

from SALib.sample import sobol as sobol_sampler
from SALib.analyze import sobol as sobol_analyzer

from sklearn.metrics import r2_score

from autoemulate.experimental_design import LatinHypercube
from autoemulate.simulations.flow_functions import FlowProblem

### Define your simulator

This simulator simulates a vessel divided to 10 compartments. 

##### Parameters
The simulation parameters include : 

1. R (Resistance):
Represents the resistance to blood flow in blood vessels, akin to the hydraulic resistance caused by vessel diameter and blood viscosity (Analogous to electrical resistor).
2. L (Inductance):
Represents the inertial effects of blood flow, capturing how blood resists changes in its velocity (Analogous to electrical inductor).
3. C (Capacitance):
Represents the compliance or elasticity of blood vessels, primarily large arteries, which store and release blood volume with changes in pressure (Analogous to a capacitor).


##### Boundary conditions 

1. Neumann boundary condition : Specifies the derivative of the variable at the boundary.
2. Dirichlet Boundary condition : Specifies the value of the variable directly at the boundary.

##### The setup 
The input flow rate in each compartment is $Q_i(t)$ for the $i^{th}$ compartment and the output flow rate is  $Q_{i+1}(t)$. 

$Q_0(t) = 
\begin{cases} 
A \cdot \sin^2\left(\frac{\pi}{t_d} t\right), & \text{if } 0 \leq t < T, \\
0, & \text{otherwise}.
\end{cases}$

Where:
- $Q_0(t)$  is the input pulse function (flow rate) at time  t 
- A   is the amplitude of the pulse 
- $t_d$  is the pulse duration.



In [1]:
fp = FlowProblem(ncycles=10, ncomp=10, amp=900.)
fp.generate_pulse_function()


NameError: name 'FlowProblem' is not defined

### solve 
Pressure in Each Compartment ($P_i$): This determines how the pressure in each compartment evolves over time, based on the inflow ($Q_i(t)$) and the outflow ($Q_{i+1}(t)$). where $i$ is the number of compartment. 




<img src="../../misc/circuit.jpg" alt="Circuit Diagram" style="width:60%;"/>

$\frac{dP_i}{dt} = \frac{1}{C_n} \left( Q_i(t) - Q_{i+1}(t) \right)$ where, $C_n = \frac{C}{n_\text{comp}}$

Flow rate equation ($Q_i$): This governs how the flow in each compartment changes over time, depending on the pressures in the neighboring compartments and the resistance and inertance properties of each compartment.

$\frac{dQ_i}{dt} = \frac{1}{L_n} \left( P_i - P_{10} - R_n Q_i(t) \right)$, where $L_n = \frac{L}{n_\text{comp}}, \quad R_n = \frac{R}{n_\text{comp}}$

In [ ]:
fp.solve()

### Plotting Pressure and flow rate 

Here the pressure and flow rate have been plotted for all 10 compartments (the colour of the line fades for later compartments). As it is apaprent from the figure the peak pressure as well as the flow rate drops for the later compartments. 

In [ ]:
fig, ax = fp.plot_res()
plt.show()

### Experimental Design

We set up the parameters for the simulation and their range. The range is set to the domain of interest for the problem we are solving. Furthermore, it is important to restrict the range to enhance the fitting of the model. 



In [ ]:
## specify valid parameter ranges
# Dictionary with parameters and their scaled ranges for the blood flow model
parameters_range = {
    'T': tuple(np.array([0.5, 1.5]) * 1.0),  # Cardiac cycle period (s)
    'td': tuple(np.array([0.8, 1.2]) * 0.2),  # Pulse duration (s)
    'amp': tuple(np.array([0.8, 1.2]) * 900.0),  # Amplitude (e.g., pressure or flow rate)
    'dt': tuple(np.array([0.5, 1.5]) * 0.001),  # Time step (s)
    'C': tuple(np.array([0.8, 1.2]) * 38.0),  # Compliance (unit varies based on context)
    'R': tuple(np.array([0.8, 1.2]) * 0.06),  # Resistance (unit varies based on context)
    'L': tuple(np.array([0.8, 1.2]) * 0.0017),  # Inductance (unit varies based on context)
    'R_o': tuple(np.array([0.8, 1.2]) * 0.025),  # Outflow resistance (unit varies based on context)
    'p_o': tuple(np.array([0.9, 1.1]) * 10.0)  # Initial pressure (unit varies based on context)
}

# Output the dictionary for verification

parameters_range

### Sampling parameters

Secondly we sample the parameter space usning the Latin hypercube sampling (LHS) method. 

Latin Hypercube Sampling (LHS) is a statistical method used for generating a sample of plausible values from a multidimensional distribution and guarantees that every sample is drawn from a different quantile of the underlying distribution. It is particularly effective for uncertainty quantification, sensitivity analysis, and simulation studies where the goal is to explore the behavior of a system or a model across a wide range of input parameter values.

In [ ]:
## sample from parameter range

N_samples = 60
lhd = LatinHypercube(parameters_range.values())
sample_array = lhd.sample(N_samples)
sample_df    = pd.DataFrame(sample_array, columns=parameters_range.keys())  
print("Number of parameters", sample_df.shape[1], "Number of samples from each parameter", sample_df.shape[0])
sample_df.head()

### Enforcing relationship between correlated parameters 

The parameters are randomly sampled from the parameter space. However, some of the parameters are by nature correlated. For example in our simulation the duration of the pulse $t_d$ is directly correlated with $T$ which is the length of a full cycle. In this cell, we enforce the relationship between these two parameters. In otherwords scale $t_d$ in accordance with the value of $T$.

In [ ]:
# enforce parameter relations (eg td <= T)
sample_df['td'] = sample_df.apply(lambda row: row['td']* row['T'], axis=1)


### Simulate

Here we run the simulation which involves:  
1- Generating the pulse functions.  
2- Solving the differential equations.  
3- Returning the pressure and flow rate in y for every set of parameters sampled (every row of sample_df data frame).

The full output of the simulation is quite large for fitting an emulator model. Therefore, it is standard practice to grab the most relavant information for training the emulator. Here we have chosen to train the emulator on the maximum pressure in each compartment. There are a variety of outputs that could be considered (max / min / average /mean / median / gradiant / PCA ).




In [ ]:
# Fixed parameters: Number of compartments and cycles
ncomp = 10
ncycles = 10

# Function to run a simulation for a given set of parameters
def simulate(param_dict):
    fp = FlowProblem(ncycles=ncycles, ncomp=ncomp, **param_dict)
    fp.generate_pulse_function()
    fp.solve()
    return fp, fp.res.t, fp.res.y

Y = []
# Iterate over each sample of parameters
for index, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
    param_dict = row.to_dict() 
    fp, t, y = simulate(param_dict) 
    # extract peak pressure
    peak_pressure = y[:ncomp, :].max()
    Y.append(peak_pressure)

### Emulate & Validate

Here we setup Autoemulate. 

The models currently available for performing cross-validation are as follows: 
- sop: SecondOrderPolynomial
- rbf: RadialBasisFunctions
- rf: RandomForest
- gb: GradientBoosting
- lgbm: LightGBM
- svm: SupportVectorMachines
- gp: GaussianProcess
- cnp: ConditionalNeuralProcess
- acnp: AttentiveConditionalNeuralProcess
- gpmt: GaussianProcessMT
- gps: GaussianProcessSklearn
- nns: NeuralNetSk

We have selected four of these models for cross-validation, feel free to explore more!  

In [ ]:
em = AutoEmulate()
parameter_names = list(parameters_range.keys())
em.setup(sample_df[parameter_names], Y, models = ['gp','cnp'])
best_model = em.compare()


In [ ]:
em.summarise_cv()

In [ ]:
em.plot_eval(model=best_model)

In [ ]:
em.evaluate(best_model)  
best_emulator = em.refit(best_model)

### Sensitivity Analysis

1. Define the problem by creating a dictionary which contains the names and the boundaries of the parameters 
2. contribution of each parameter , importance of each parameter 
    - S₁: First-order sensitivity index.
    - S₂: Second-order sensitivity index.
    - Sₜ: Total sensitivity index.

In [ ]:
# Extract parameter names and bounds from the dictionary
parameter_names = list(parameters_range.keys())
parameter_bounds = list(parameters_range.values())

# Define the problem dictionary for Sobol sensitivity analysis
problem = {
    'num_vars': len(parameter_names),
    'names': parameter_names,
    'bounds': parameter_bounds
}


In [ ]:
em.sensitivity_analysis(problem=problem)


In [ ]:

sa_values = sobol_sampler.sample(problem=problem, 
                                     N=1000, #'Number of values sampled for the Sobol sensitivity analysis
                                     calc_second_order=True
                                     )
print('sa_values shape\n', sa_values.shape)
em_predict = best_emulator.predict(sa_values)
print('em_values shape\n', em_predict.shape)

In [ ]:
# Perform Sobol sensitivity analysis
sensitivity_results = sobol_analyzer.analyze(
    problem=problem,
    Y=em_predict,
    calc_second_order=True,
    print_to_console=True
)

In [ ]:

# Extracting first-order, second-order, and total-order sensitivity indices
print(sensitivity_results['S1'])  # First-order sensitivity indices
print(sensitivity_results['ST']) # Total-order sensitivity indices
print(sensitivity_results['S2'])  # Second-order sensitivity indices matrix


In [ ]:
Si = sensitivity_results['S1']
parameter_names = problem['names'] 

# Si should be a 2D array for seaborn's heatmap, so we reshape it accordingly
Si_reshaped = Si.reshape(1, -1)  # 1 row, len(Si) columns

# plot parameter heatmap (makes more sense for multiple components)
fig, ax = plt.subplots(figsize=(0.2 * len(Si) + 5, 1))  # Adjust the size as needed
cmap = plt.get_cmap('inferno').reversed()
sb.heatmap(Si_reshaped, vmin=0.0, vmax=1.0, ax=ax, linecolor='k', linewidths=2, cmap=cmap, annot=True)

ax.set_xticks(np.arange(len(parameter_names)) + 0.5)
ax.set_xticklabels(parameter_names, rotation=45, ha='right')
ax.set_yticks([])  # No need for y-axis labels since we only have one row

# Tight layout for better fit
fig.tight_layout()

# Show the plot
plt.show()

In [ ]:
#visualize most relevant parameters
param_1 = sample_df['amp']
param_2 = sample_df['R']

plt.scatter(param_1, param_2, c=Y, cmap='viridis')
plt.xlabel('param 1')
plt.ylabel('param 2')
plt.colorbar(label="peak pressure")
plt.show